In [4]:
import torch
import os
import ignite
import matplotlib.pyplot as plt
from tqdm import tqdm
import pytorch_ssim
from ignite.metrics import SSIM, PSNR
import glob
import numpy as np
from monai.networks.nets import UNet
import monai
from monai import transforms
from monai.utils import set_determinism, first
from sklearn.metrics import mean_squared_error as mse

from monai.transforms import(
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    LoadImaged,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandSpatialCropSamplesd,
    RandShiftIntensityd,
    RandZoomd,
    ScaleIntensityd,
    Spacingd,
    SpatialPadd,
    GaussianSmoothd,
    RandRotate90d,
    ToTensord,
    RandSpatialCropd,
    RandGaussianSmoothd,    
    RandGaussianSharpend,
    RandGaussianNoise,
    
)
import nibabel as nib

from monai.data import (
    DataLoader,
    Dataset,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai.metrics import MSEMetric as mse
from monai.metrics import MAEMetric as mae
from monai.metrics import PSNRMetric as psnr
from monai.metrics import SSIMMetric as ssim
from monai.metrics import RMSEMetric as rmse

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print(device)

cuda:0


In [ ]:
data_dir3

In [8]:
data_dir =  glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_*.nii.gz')
data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/low_T2_*.nii.gz')
data_dir3 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_*.nii.gz')
valid_ind = np.arange(0, 6)
#valid_ind = np.arange(25, 34)
data_dicts = [
    {
        'image' : data_dir[i],
        'low_res_image': data_dir2[i],
        'syn' : data_dir3[i]
         
    }for i in valid_ind
]
valid_data = data_dicts

In [ ]:
val_transforms = transforms.Compose(
    [
          transforms.LoadImaged(keys=["image",'low_res_image','syn']),
        transforms.EnsureChannelFirstd(keys=["image","low_res_image",'syn']),
        transforms.ScaleIntensityd(
                keys=["image",'low_res_image','syn'],
                minv=0.0,
                maxv=1.0,
        ),           
#         transforms.SpatialPadd(keys=['image','low_res_image'],spatial_size=(256,256,256)),
        transforms.CenterSpatialCropd(keys=['image','low_res_image','syn'],roi_size = (192,220,192)),
    ]
)
val_ds = CacheDataset(data=valid_data, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=True, num_workers=4)

In [ ]:
check_data = first(val_loader)
fig, ax = plt.subplots(nrows=1,ncols=3)
for i in range(1):
    plt.subplot(131)
    plt.title('image')
    plt.imshow(check_data['image'][i,0,:,30,:],cmap = 'gray')
    plt.subplot(132)
    plt.title('low res image')
    plt.imshow(check_data['low_res_image'][i,0,:,30,:],cmap = 'gray')
    plt.axis('off')
    plt.subplot(133)
    plt.title('syn image')
    plt.imshow(check_data['syn'][i,0,:,30,:],cmap = 'gray')
    plt.axis('off')

    plt.show()

In [ ]:
from models.network_swinir import SwinIR as net

model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=7,
        out_channels=1,
        channels=(64, 128, 256, 512),
        strides=(2, 2, 2),
        num_res_units=3,
    ).to(device)

model.load_state_dict(torch.load(os.path.join("./swinIR_weight/SwinUNetIR_v3.pth")))
# model.load_state_dict(torch.load(os.path.join("./High_res_UNet_v5_2.pth")))

### PSNR

In [3]:
import torch
import torchvision
import numpy as np
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.metrics import mean_absolute_error as mae


### SSIM

In [ ]:
# 위에서 import한 라이브러리들을 이용하여 계산


model.eval()
psnr_value = []
ssim_value = []
mse_value = []
with torch.no_grad():
    for step, batch in tqdm(enumerate(val_loader)):
        # 기존 코드
        rand = 30
        iamges = batch['low_res_image'].to(device)
        target = batch['image'].to(device)
        val_output = batch['syn'].to(device)
        
#         images = batch['low_res_image']
#         images = images.squeeze(1)
#         images = images[...,rand:rand+7, :].to(device)
#         images = images.permute(0,2,1,3)
#         target = batch['image'][..., rand+3, :].to(device)
        
        
        print(images.shape)
        print(target.shape)
        print(val_output.shape)
        
#         plt.subplot(141)
#         plt.imshow(images[0,3,...].detach().cpu(),cmap='gray')
#         plt.axis('off')
#         plt.title('input')

#         plt.subplot(142)
#         plt.imshow(target[0,0,...].detach().cpu(),cmap='gray')
#         plt.axis('off')
#         plt.title('gt')

#         plt.subplot(143)
#         plt.imshow(val_output[0,0,:,rand,:].detach().cpu(),cmap='gray')
#         plt.title('prediction')
#         plt.axis('off')
        
#         k = target[0,0,...]-val_output[0,...]
#         k[k<0.01] = 0
#         plt.subplot(144)
#         plt.imshow(k.detach().cpu(),cmap='gray')
#         plt.title('diff')
#         plt.axis('off')
# #         plt.colorbar()
        
        
#         plt.show()

#         val_output = val_output.cpu().numpy().squeeze()
#         target = target.cpu().numpy().squeeze()
# #         print(target.shape, val_output.shape)
        

#         # 모델 출력과 실제 타겟 이미지를 numpy 배열로 변환
#         ssims = ssim(target,val_output[0,...],data_range = val_output.max()-val_output.min())
#         psnrs = psnr(target,val_output[0,...],data_range = val_output.max()-val_output.min())
#         maes = mae(target,val_output[0,...])
#         print(maes)
#         print(ssims)
#         print(psnrs)
#         # 각 이미지 별로 PSNR 및 SSIM 계산
# #         for i in range(len(val_output)):
           
#         psnr_value.append(psnrs)
#         ssim_value.append(ssims)
#         mse_value.append(maes)

# # # 결과 출력
# average_psnr = sum(psnr_value)/len(psnr_value)
# average_ssim = sum(ssim_value)/len(ssim_value)
# average_mse = sum(mse_value)/len(mse_value)
# print(f"Average PSNR: {average_psnr}")
# print(f"Average SSIM: {average_ssim}")
# print(f"Average MSE : {average_mse}")
#     nifti_img = nib.load('/home/hufsbme/EA/bcp_t2_swinIR.nii.gz')
#     image_data = nifti_img.get_fdata()
#     low_res_nifti = nib.Nifti1Image(val_output, nifti_img.affine, nifti_img.header)

#     nib.save(low_res_nifti,'/home/hufsbme/EA/test.nii.gz')


In [6]:
import cv2
import torch
import torch.nn.functional as F
from skimage.metrics import structural_similarity as ssim
from monai.metrics import PSNRMetric as psnr
import glob

data_dir =  glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_*.nii.gz')
data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/low/low_T2_*.nii.gz')
data_dir3 = glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_*_6.nii.gz')

k = mse()
psnrs = psnr(max_val=1.0)
# ssims = ssim(data_range=1.0)
mse_value= []
psnr_value = []
ssim_value = []

for i in range(len(data_dir)):
    image1 = nib.load(data_dir[i])
    image1 = image1.get_fdata()#216,101,208
    image2 = nib.load(data_dir3[i])
    image2 = image2.get_fdata()#216,101,208
    
    print(data_dir[i])
    print(data_dir3[i])
    
    print(k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    print(psnrs(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    s = ssim(image2,image1,data_range = image2.max()-image2.min())
    print(s)
#     print(SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    m = k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    p = psnrs(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    
#     s = SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    
    mse_value.append(m)
    psnr_value.append(p)
    ssim_value.append(s)

mean_mse = sum(mse_value)/len(mse_value)
mean_psnr = sum(psnr_value)/len(psnr_value)
mean_ssim = sum(ssim_value)/len(ssim_value)

print(mean_mse,mean_psnr,mean_ssim)

/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_0.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_0_6.nii.gz
tensor([[0.0005]])
tensor([[32.7277]])
0.8026460354007678
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_1.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_1_6.nii.gz
tensor([[0.0005]])
tensor([[32.7509]])
0.8288361178536099
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_2.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_2_6.nii.gz
tensor([[0.0004]])
tensor([[33.5409]])
0.7777278951364791
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_3.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_3_6.nii.gz
tensor([[0.0006]])
tensor([[32.2008]])
0.8910917651913712
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_4.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_4_6.nii.gz
tensor([[0.0073]])
tensor([[21.3527]])

In [7]:
import cv2
import torch
import torch.nn.functional as F
from skimage.metrics import structural_similarity as ssim
from monai.metrics import PSNRMetric as psnr

data_dir =  glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/origin/T2_*.nii.gz')
data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/low/low_T2_*.nii.gz')
data_dir3 = glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_*_6.nii.gz')

k = mse()
psnrs = psnr(max_val=1.0)
# ssims = ssim(data_range=1.0)
mse_value= []
psnr_value = []
ssim_value = []

for i in range(len(data_dir)):
    image1 = nib.load(data_dir[i])
    image1 = image1.get_fdata()#216,101,208
    image2 = nib.load(data_dir3[i])
    image2 = image2.get_fdata()#216,101,208
    
    print(data_dir[i])
    print(data_dir3[i])
    
    print(k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    print(psnrs(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    s = ssim(image2,image1,data_range = image2.max()-image2.min())
    print(s)
#     print(SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    m = k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    p = psnrs(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    
#     s = SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    
    mse_value.append(m)
    psnr_value.append(p)
    ssim_value.append(s)

mean_mse = sum(mse_value)/len(mse_value)
mean_psnr = sum(psnr_value)/len(psnr_value)
mean_ssim = sum(ssim_value)/len(ssim_value)

print(mean_mse,mean_psnr,mean_ssim)

/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/origin/T2_0.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_0_6.nii.gz
tensor([[0.0007]])
tensor([[31.6232]])
0.8079457212537325
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/origin/T2_1.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_1_6.nii.gz
tensor([[0.0003]])
tensor([[35.8968]])
0.8648696736773105
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/origin/T2_2.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_2_6.nii.gz
tensor([[0.0005]])
tensor([[33.2229]])
0.8210928785131121
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/origin/T2_3.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_3_6.nii.gz
tensor([[0.0006]])
tensor([[32.1029]])
0.8946014097249072
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/origin/T2_4.nii.gz
/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_4_6.nii.gz
tensor([[0.0038]])
tensor([[24.1683]])
0.8256992905789317


In [37]:
import cv2
import torch
import torch.nn.functional as F
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

data_dir =  glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_*.nii.gz')
data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/low_T2_*.nii.gz')
data_dir3 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_*.nii.gz')

k = mse()
# psnrs = psnr(max_val=1.0)
# ssims = ssim(data_range=1.0)
mse_value= []
psnr_value = []
ssim_value = []

for i in range(len(data_dir)):
    image1 = nib.load(data_dir[i])
    image1 = image1.get_fdata()#216,101,208
    image2 = nib.load(data_dir3[i])
    image2 = image2.get_fdata()#216,101,208
    
    print(data_dir[i])
    print(data_dir3[i])
    
    print(k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    s = ssim(image2,image1,data_range = image2.max()-image2.min())
    print(s)
#     print(SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    m = k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    p = psnr(image2,image1,data_range = image2.max()-image2.min())
    print(p)
#     s = SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    
    mse_value.append(m)
    psnr_value.append(p)
    ssim_value.append(s)

mean_mse = sum(mse_value)/len(mse_value)
mean_psnr = sum(psnr_value)/len(psnr_value)
mean_ssim = sum(ssim_value)/len(ssim_value)

print(mean_mse,mean_psnr,mean_ssim)

/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_25.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_25.nii.gz
tensor([[0.0024]])
0.80619912996896
26.311794591600083
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_24.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_24.nii.gz
tensor([[0.0038]])
0.6324213341396453
20.856412299728934
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_23.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_23.nii.gz
tensor([[0.0033]])
0.765459576911232
24.329522063313323
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_22.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_22.nii.gz
tensor([[0.0050]])
0.7823052070570302
22.83518511610223
tensor([[0.0036]]) 23.583228517686145 0.746596312019217


In [40]:
import cv2
import torch
import torch.nn.functional as F
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

data_dir =  glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/T2_*.nii.gz')
data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir//low_T2_*.nii.gz')
data_dir3 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir//syn_T2_*.nii.gz')

k = mse()
# psnrs = psnr(max_val=1.0)
# ssims = ssim(data_range=1.0)
mse_value= []
psnr_value = []
ssim_value = []

for i in range(len(data_dir)):
    image1 = nib.load(data_dir[i])
    image1 = image1.get_fdata()#216,101,208
    image2 = nib.load(data_dir3[i])
    image2 = image2.get_fdata()#216,101,208
    
    print(data_dir[i])
    print(data_dir3[i])
    
    print(k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    s = ssim(image2,image1,data_range = image2.max()-image2.min())
    print(s)
#     print(SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0)))
    m = k(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    p = psnr(image2,image1,data_range = image2.max()-image2.min())
    print(p)
#     s = SSIM(y_pred=torch.Tensor(image1).unsqueeze(0), y=torch.Tensor(image2).unsqueeze(0))
    
    mse_value.append(m)
    psnr_value.append(p)
    ssim_value.append(s)

mean_mse = sum(mse_value)/len(mse_value)
mean_psnr = sum(psnr_value)/len(psnr_value)
mean_ssim = sum(ssim_value)/len(ssim_value)

print(mean_mse,mean_psnr,mean_ssim)

/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/T2_25.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/syn_T2_25.nii.gz
tensor([[0.0022]])
0.8275148311798214
25.44016202784013
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/T2_24.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/syn_T2_24.nii.gz
tensor([[0.0035]])
0.6907297586807659
17.95475810622414
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/T2_23.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/syn_T2_23.nii.gz
tensor([[0.0024]])
0.7593007468162184
25.588175609997258
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/T2_22.nii.gz
/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/swinir/syn_T2_22.nii.gz
tensor([[0.0048]])
0.7976494040181168
21.99474917878483
tensor([[0.0032]]) 22.74446123071159 0.7687986851737305


In [14]:
image2.shape

(192, 101, 192)

In [10]:
import cv2
import torch
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error as mse
data_dir =  glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/T2_*.nii.gz')
data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/low_T2_*.nii.gz')
data_dir3 = glob.glob('/nfs/hufsaims/kjh_shared/euna/data_bcp/improvement_result/unet/syn_T2_*.nii.gz')

# SSIM (Structural Similarity Index) 계산 함수
def calculate_ssim(image1, image2):
    return ssim(image1, image2)

# PSNR (Peak Signal-to-Noise Ratio) 계산 함수
def calculate_psnr(image1, image2):
    mse = torch.mean((image1 - image2) ** 2)
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

# MSE (Mean Squared Error) 계산 함수
def calculate_mse(image1, image2):
    return torch.mean((image1 - image2) ** 2)
ss = []
psnr_s = []
mse_s = []

for i in range(len(data_dir)):
    image1 = nib.load(data_dir[i])
    image1 = image1.get_fdata()#216,101,208
    image2 = nib.load(data_dir3[i])
    image2 = image2.get_fdata()#216,101,208
    

    
#     ssim_score = calculate_ssim(image1, image2)
#     psnr_score = calculate_psnr(image1, image2)
    mse_score = calculate_mse(image1, image2)
    ssims = SSIM(image1,image2,data_range = val_output.max()-val_output.min())
    psnrs = PSNR(image1,image2,data_range = val_output.max()-val_output.min())
    mses = calculate_mse(image1,image2)

    ss.append(ssims)
    psnr_s.append(psnrs)
    mse_s.append(mses)


avg_ss = sum(ss)/len(ss)
avg_psnr = sum(psnr_s)/len(psnr_s)
avg_mse = sum(mse_s)/len(mse_s)

print(f'SSIM: {avg_ss:.4f}')
print(f'PSNR: {avg_psnr:.4f} dB')
print(f'MSE: {avg_mse:.4f}')


TypeError: mean(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
with torch.no_grad():
    for step, batch in tqdm(enumerate(val_loader)):
        images = batch['image']
        images = images.squeeze(1)
        k = 192/2
        plt.figure(dpi = 200)
        plt.subplot(1,4,1)
        plt.imshow(batch['image'][0,0, :, 140, :],cmap = 'gray')
        plt.subplot(1,4,2)
        plt.imshow(batch['image'][0,0, :, 105, :],cmap = 'gray')
        plt.subplot(1,4,3)

        plt.imshow(batch['image'][0,0,:,90, :],cmap = 'gray')
        plt.subplot(1,4,4)

        plt.imshow(batch['image'][0,0,:,65],cmap = 'gray')
        plt.show()
        print(images.shape)
        plt.imshow(batch['image'][0,0,:,81,:],cmap='gray')
        plt.show()

## interpolation과 비교하기

In [ ]:
with torch.no_grad():
    for step, batch in tqdm(enumerate(val_loader)):
        rand = 80
        images = batch['low_res_image']
        images = images.squeeze(1)
        images = images[...,rand:rand+7, :].to(device)
        images = images.permute(0,2,1,3)
        target = batch['image'][..., rand+3, :].to(device)
        val_output = model(images)
#         print(images.shape)
        nearest = torch.nn.functional.interpolate(images,size=(192,192),mode = 'nearest')
        bilinear = torch.nn.functional.interpolate(images,size=(192,192),mode = 'bilinear')
        bicubic = torch.nn.functional.interpolate(images,size=(192,192),mode = 'bicubic')
#         linear = torch.nn.functional.interpolate(images,size=(192,192),mode = 'linear')
        plt.figure(figsize=(15,3))

        plt.subplot(141)
        plt.imshow(val_output[0,0,...].detach().cpu(),cmap='gray')
        plt.title('prediction')
        plt.axis('off')
        
        plt.subplot(142)
        plt.imshow(nearest[0,3,...].detach().cpu(),cmap='gray')
        plt.title('Nearest')
        plt.axis('off')

        
        plt.subplot(143)
        plt.imshow(bilinear[0,3,...].detach().cpu(),cmap='gray')
        plt.title('Bilinear')
        plt.axis('off')

        
        
        plt.subplot(144)
        plt.imshow(bicubic[0,3,...].detach().cpu(),cmap='gray')
        plt.title('Bicubic')
        plt.axis('off')

        plt.show()
        k_1  = target-val_output[0,3,...]
        
        k_1[k_1<0.01]=0

        k_2 = target - nearest[0,3,...]

        k_2[k_2<0.01]=0

        k_3 = target - bilinear[0,3,...]

        k_3[k_3<0.01]=0

        k_4 = target - bicubic[0,3,...]
                               

        k_4[k_4<0.01]=0
        
        
        plt.figure(figsize=(15,3))
        
        plt.subplot(1,4,1)
        plt.imshow((np.abs(k_1[0,0,...].detach().cpu())/(target.max()-target.min())), cmap = 'jet', vmax = 0.1)
        plt.colorbar()
        plt.axis('off')
       
        plt.subplot(142)
        plt.imshow((np.abs(k_2[0,0,...].detach().cpu())/(target.max()-target.min())), cmap = 'jet', vmax = 0.1)
        plt.colorbar()
        plt.axis('off')
        
        plt.subplot(143)
        plt.imshow((np.abs(k_3[0,0,...].detach().cpu())/(target.max()-target.min())), cmap = 'jet', vmax = 0.1)
        plt.colorbar()
        plt.axis('off')
        
        plt.subplot(144)
        plt.imshow((np.abs(k_4[0,0,...].detach().cpu())/(target.max()-target.min())), cmap = 'jet', vmax = 0.1)
        plt.colorbar()
        plt.axis('off')
        
        plt.show()
        
        plt.subplot

        

In [ ]:
k = target-val_output[3,...]

In [ ]:
0~1 1%

In [ ]:
k[k<0.01]=0
plt.imshow(k)

In [ ]:
test_dict = [
{
    'image' : '/home/hufsbme/EA/T2.nii.gz'
}      
    ] 
val_transforms = transforms.Compose(
    [
          transforms.LoadImaged(keys=["image"]),
        transforms.EnsureChannelFirstd(keys=["image"]),
        transforms.ScaleIntensityd(
                keys=["image"],
                minv=0.0,
                maxv=1.0,
        ),           
#         transforms.SpatialPadd(keys=['image','low_res_image'],spatial_size=(256,256,256)),
#         transforms.CenterSpatialCropd(keys=['image'],roi_size = (192,192,192)),
    ]
)
test_ds = CacheDataset(data=test_dict, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=True, num_workers=4)

In [ ]:

plt.imshow(images[0,4,...].detach().cpu(),cmap='gray')
plt.axis('off')
plt.title('input')

In [ ]:

model.eval()
psnr_value = []
ssim_value = []
with torch.no_grad():
    for step, batch in tqdm(enumerate(test_loader)):
        # 기존 코드
        rand = 80
        images = batch['image']
        images = images.squeeze(1)
        images = images[...,rand:rand+7, :].to(device)
        print(images.shape)
        images = images.permute(0,2,1,3)
        target = batch['image'][..., rand+3, :].to(device)
        
        val_output = model(images)
        plt.subplot(121)
        plt.imshow(images[0,4,...].detach().cpu(),cmap='gray')
        plt.axis('off')
        plt.title('input')

#         plt.subplot(132)
#         plt.imshow(target[0,0,...].detach().cpu(),cmap='gray')
#         plt.axis('off')
#         plt.title('gt')

        plt.subplot(122)
        plt.imshow(val_output[0,0,...].detach().cpu(),cmap='gray')
        plt.title('prediction')

        plt.axis('off')
        plt.show()
        val_output = val_output.cpu().numpy().squeeze()
        target = target.cpu().numpy().squeeze()

        

#         모델 출력과 실제 타겟 이미지를 numpy 배열로 변환
        ssims = ssim(target,val_output,data_range = val_output.max()-val_output.min())
        psnrs = psnr(target,val_output,data_range = val_output.max()-val_output.min())
        print(ssims)
        print(psnrs)
        # 각 이미지 별로 PSNR 및 SSIM 계산
#         for i in range(len(val_output)):
           
        psnr_value.append(psnrs)
        ssim_value.append(ssims)

# # 결과 출력
average_psnr = sum(psnr_value)/len(psnr_value)
average_ssim = sum(ssim_value)/len(ssim_value)
print(f"Average PSNR: {average_psnr}")
print(f"Average SSIM: {average_ssim}")

In [ ]:

import nibabel as nib
nifti_img = nib.load('/home/hufsbme/EA/T2.nii.gz')
image_data = nifti_img.get_fdata()
low_res_nifti = nib.Nifti1Image(val_output, nifti_img.affine, nifti_img.header)

nib.save(low_res_nifti,'/home/hufsbme/EA/test.nii.gz')